In [1]:
#   |'''''''''''''╔╬╬╬╬╬╬╬╬   _____  _____      _____  _____      ___   __
#   |            ╔╬╬╬╬╬╬╬╬╬  |\   _ \  _  \    |\   _ \  _  \    |\  \|\  \
#   | ░░         ╬╬╬╬╬╬╬╬╬╬  \ \  \\__\ \  \   \ \  \\__\ \  \   \ \  \/  /|_
#    ░░░░        ╬╬╬╬╬╬╬╬╬╬   \ \  \|__| \  \   \ \  \|__| \  \   \ \   ___  \
#   ░░░░░╦╬╦    ╔╬╬╬╬╬╬╬╬╬╬    \ \  \   \ \  \   \ \  \   \ \  \   \ \  \ \   \
#  ░░░░░╬╬╬╬ ▓▓└╬╬╬╬╬╬╬╬╬╬╬     \ \__\   \ \__\   \ \__\   \ \__\   \ \__\ \___\
# ░░░░░╔╬╬╬ ▓▓▓  ╓╬╬╬╬╬╬╬╬╬      \|__|    \|__|    \|__|    \|__|    \|__| \|__|
# ░░░░░╠╬╬╬ ▓▓▓  └╬╬╬╬╬╬╬╬╬
#  ░░░░└╬╬╬╬ ▓▓   ╬╬╬╬╬╬╬╬╬  Lehrstuhl für Mensch-Maschine-Kommunikation
#  ░░░░░╙╬╬╬╩            ╬╬  Technische Universität München
#   ░░░░░░╚ '''''''''''''''  Author: Tobias Watzel
#    ░░░                     Copyright 2020
#

%matplotlib widget
import cv2
import glob
import re
import io
import ipywidgets as widgets
import numpy as np
from IPython.display import clear_output, Image, display
import PIL.Image
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib as mpl
from matplotlib import cm
from functools import partial
import dill

## Versuchsbeschreibung
Dieser Versuch soll Ihnen ein Gefühl für verschiedene Farbräume geben. Wählen Sie ein Bild aus und lassen Sie sich die verschiedenen Farbkomponenten dieses Bildes anzeigen.

In [5]:
# create dict with name of image and its path (FIX? hacked...)
dict_images = {entry.replace('/',' ').replace('.',' ').split()[1]:entry for entry in sorted(glob.glob("images/*"))}
scaling_dict = {'down_x': 0.5, 'down_y': 0.5, 'up_x': 2.0, 'up_y': 2.0}

def convert_image(a, fmt='png'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    return f.getvalue()
    
    
def update_image():  
    # read image
    image = cv2.imread(dict_images[select.value])
    
    # image in RGB (just switching dims)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    box.children[1].value = convert_image(image_rgb)
    box.children[1].width = image_rgb.shape[1]
    box.children[1].height = image_rgb.shape[0]
    
    # scale down image
    image_down = cv2.resize(image_rgb, None, fx=scaling_dict['down_x'], fy=scaling_dict['down_y'])
    box.children[3].children[0].value = convert_image(image_down)
    box.children[3].children[0].width = image_down.shape[1]
    box.children[3].children[0].height = image_down.shape[0]
    
    # scale up image
    image_up = cv2.resize(image_down, None, fx=scaling_dict['up_x'], fy=scaling_dict['up_y'])
    box.children[5].children[0].value = convert_image(image_up)
    box.children[5].children[0].width = image_up.shape[1]
    box.children[5].children[0].height = image_up.shape[0]

def change_val(change):
    if change['type'] == 'change' and change['name'] == 'value':
        scaling_dict[change['owner'].description.replace('.', '_').lower()] = change['owner'].value
        update_image()
        
        
def create_display(scaling = 'up'):
    display_list = []
    init_image = cv2.cvtColor(cv2.imread('images/birne.png'), cv2.COLOR_BGR2RGB)
    
    image = widgets.Image(value=convert_image(init_image), format='png', width=300, height=400)
    helper_dict = {}
    for axes in ['x', 'y']:
        helper_dict['label' + axes] = widgets.Label(value='Skalierungsfaktor in %s:' % axes.upper())
        if scaling == 'up':
            helper_dict[axes] = widgets.FloatSlider(value=0.5, min=1.0, max=2.0, step=0.05, disabled=False, continuous_update=False, orientation='horizontal',
                                                      readout=True, readout_format='.2f', description = ('%s.' + axes.upper()) % scaling)
            helper_dict[axes].value = scaling_dict['%s_%s' % (scaling, axes)]
        else:
            helper_dict[axes] = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, disabled=False, continuous_update=False, orientation='horizontal',
                                                      readout=True, readout_format='.2f', description = ('%s.' + axes.upper()) % scaling)
            helper_dict[axes].value = scaling_dict['%s_%s' % (scaling, axes)]
        helper_dict[axes].observe(change_val)
    
    return widgets.HBox([image, widgets.VBox(list(helper_dict.values()))])

# create box
label_1 = widgets.Label(value=r'\(\textbf{Originalbild}\)')
label_2 = widgets.Label(value=r'\(\textbf{Verkleinertes Bild}\)')
label_3 = widgets.Label(value=r'\(\textbf{Vergrößertes Bild}\)')
init_image = cv2.cvtColor(cv2.imread('images/birne.png'), cv2.COLOR_BGR2RGB)
shape_image = init_image.shape
image_1 = widgets.Image(value=convert_image(init_image), format='png', width=init_image.shape[0], height=init_image.shape[1])
box = widgets.VBox([label_1, image_1, label_2, create_display('down'), label_3, create_display('up')])

select = widgets.Select(
    options=list(dict_images.keys()),
    value='birne',
    description='Dateiliste:')

select.observe(change_val)

display(select)
display(box)

update_image()

Select(description='Dateiliste:', options=('birne', 'buch', 'fahrzeug', 'klammermaschine', 'mandrill', 'orange…

### 1. Ein .tif-Bild habe 10MB. Wieviel MB hat das skalierte Bild, wenn der Skalierungsfaktor in x- und y-Richtung jeweils 50% beträgt?


In [6]:
save_image = 'save/size_image.dill'

size_image = ''
# try to load precicion value
try: 
    with open(save_image, 'rb') as fp:
        size_image = dill.load(fp)
        if size_image == '':
            size_image = 0.0
except:
    pass

text_size_image = widgets.Text(value=str(size_image),
                              placeholder='',
                              description='',
                              disabled=False,
                              continuous_update=True)

def callback_size_image(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # save when press enter
        with open(save_image, 'wb') as fp:
            dill.dump(change['new'], fp)


text_size_image.observe(callback_size_image)

label = widgets.Label(value = 'MB (Trennen Sie die Nachkommastellen bitte durch einen Punkt ab.)')
display(widgets.HBox([text_size_image, label]))

In [7]:
# Autograding answer, please ignore
